In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from tqdm import tqdm

%load_ext lab_black

## ESRI data

The most recent 2020 census tracts are used to aggregate the data. More info about each netCDF is below:
 
- Daily Minimum temperature from 2006-2021  
- Daily Maximum temperature from 2006-2021
- Daily Relative Humidity from 2006-2021  
- Daily Smoke from 2006-2021 
- and Daily PM 2.5 data from 2006-2020

In [ ]:
tmax = xr.open_dataset("data/esri/MaxTemp_2006_2021_Cali.nc")

In [ ]:
tmax["MAX_TEMPERATURE_NONE_SPATIAL_NEIGHBORS"].plot()

In [ ]:
tmin = xr.open_dataset("data/esri/MinTemp_2006_2021_Cali.nc")

In [ ]:
rhum = xr.open_dataset("data/esri/RelHum_2006_2021_Cali.nc")

## Join tmax, tmin and rhum

In [ ]:
df = xr.merge([tmax, tmin[["MIN_TEMPERATURE_NONE_SPATIAL_NEIGHBORS"]]])

In [ ]:
df = xr.merge([df, rhum[["RELATIVE_HUMIDITY_NONE_SPATIAL_NEIGHBORS"]]])

In [ ]:
del tmin
del tmax
del rhum

In [ ]:
df = df[
    [
        "FIPS",
        "MAX_TEMPERATURE_NONE_SPATIAL_NEIGHBORS",
        "MIN_TEMPERATURE_NONE_SPATIAL_NEIGHBORS",
        "RELATIVE_HUMIDITY_NONE_SPATIAL_NEIGHBORS",
    ]
].to_dataframe()

In [ ]:
df = df.rename(
    columns={
        "MAX_TEMPERATURE_NONE_SPATIAL_NEIGHBORS": "tmax",
        "MIN_TEMPERATURE_NONE_SPATIAL_NEIGHBORS": "tmin",
        "RELATIVE_HUMIDITY_NONE_SPATIAL_NEIGHBORS": "rhum",
    }
)

In [ ]:
df.head()

In [ ]:
df = df.loc[(slice(pd.Timestamp("2006-01-01"), None), slice(None)), :]

In [ ]:
df = df.loc[(slice(None, pd.Timestamp("2020-12-31")), slice(None)), :]

In [ ]:
df.to_parquet("outputs/temp_merge_1.parquet")

## Add wildfire day

In [ ]:
df = pd.read_parquet("outputs/temp_merge_1.parquet")

In [ ]:
len(df)

In [ ]:
df = df.reset_index()

In [ ]:
df = df.groupby(by=["time", "lat", "lon"]).first()

In [ ]:
wf = xr.open_dataset("data/esri/WildfirePresence_2006_2021_CA_daily.nc")

In [ ]:
# "FIPS" start at 0, 1, 2, 3,...

In [ ]:
wf = wf["FIREDAY_NONE_ZEROS"].to_dataframe()

In [ ]:
wf = wf.rename(columns={"FIREDAY_NONE_ZEROS": "wfday"})

In [ ]:
len(wf)

In [ ]:
wf = wf.groupby(by=["time", "lat", "lon"]).first()

In [ ]:
tqdm.pandas()

In [ ]:
df = df.join(wf, rsuffix="_wf").progress_apply(lambda x: x)

In [ ]:
df.to_parquet("outputs/temp_merge_2.parquet")

In [ ]:
len(df)

In [ ]:
df.isna().sum()

## Add PM 2.5

In [ ]:
pm25 = xr.open_dataset("data/esri/PM25_Nature_2006_2020_Cali.nc")

In [ ]:
df_pm25 = pm25[["MEAN_NONE_SPATIAL_NEIGHBORS"]].to_dataframe()

In [ ]:
df_pm25 = df_pm25.rename(columns={"MEAN_NONE_SPATIAL_NEIGHBORS": "pm25"})

In [ ]:
df_pm25 = df_pm25.reset_index()

In [ ]:
df_pm25 = df_pm25.groupby(by=["time", "lat", "lon"]).first()

In [ ]:
df_pm25.head()

In [ ]:
df.head()

In [ ]:
df = df.join(df_pm25, rsuffix="_pm25").progress_apply(lambda x: x)

In [ ]:
df.to_parquet("outputs/temp_merge_3.parquet")

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
len(df.FIPS.unique())

In [ ]:
df = df.reset_index()

## Add crosswalk

In [ ]:
df = pd.read_parquet("outputs/temp_merge_3.parquet")

In [ ]:
df.head()

In [ ]:
df = df.reset_index()

In [ ]:
import geopandas as gpd

In [ ]:
gdf = gpd.read_file("data/esri/FIPSMapping.gdb")

In [ ]:
df.FIPS = df.FIPS.astype(int)

In [ ]:
gdf = gdf.drop(columns=["Shape_Length", "Shape_Area"])

In [ ]:
df = df.merge(gdf, on="FIPS")

In [ ]:
len(df)

In [ ]:
df.isna().sum()

In [ ]:
type(df)

In [ ]:
df = df.drop(columns="geometry")

In [ ]:
df.to_parquet("outputs/temp_merge_4.parquet")

## Join smoke PM2.5

In [ ]:
# smoke_df = pd.read_parquet("outputs/smoke_pm25_predicted_with_fips.parquet")

In [ ]:
smoke_df = pd.read_csv(
    "data/smoke_pm/smoke_pm_all_time_2020_map.csv", parse_dates=["date"]
)

In [ ]:
smoke_df.head()

In [ ]:
df.head()

In [ ]:
df["FIPS_1"] = df["FIPS_1"].astype(int)

In [ ]:
merged = df.merge(smoke_df, left_on=["FIPS_1", "time"], right_on=["GEOID", "date"])

In [ ]:
merged.head()

In [ ]:
len(merged)

In [ ]:
merged = merged.drop(
    columns=["lat", "lon", "locations", "FIPS", "locations_pm25", "date"]
)

In [ ]:
merged.to_parquet("outputs/final_merge_5.parquet")

In [ ]:
merged.isna().sum()

In [ ]:
gdf = gpd.read_file("data/esri/FIPSMapping.gdb")

In [ ]:
gdf.plot()

In [ ]:
temp = merged.groupby("FIPS").sum()

In [ ]:
temp = temp.reset_index()

In [ ]:
temp.dtypes

In [ ]:
gdf.dtypes

In [ ]:
temp.FIPS = temp.FIPS.astype(int)

In [ ]:
gdf = gdf.merge(temp, left_on="FIPS", right_on="FIPS")

In [ ]:
fig, ax = plt.subplots(1, 1)
gdf.plot(column="tmax", ax=ax, legend=True, linewidth=0, cmap="viridis_r")

plt.show()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(1, 1)
gdf.plot(column="smoke_pm", ax=ax, legend=True, linewidth=0, cmap="viridis_r")

plt.show()